In [1]:
from pyscal.core import System
import json
import yaml
import numpy as np
from json import JSONEncoder

In [13]:
class NumpyArrayEncoder(JSONEncoder):
    """
    Encode numpy to dump in json
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyArrayEncoder, self).default(obj)
    
def get_angle(vec1, vec2):
    """
    Get angle between two vectors in degrees
    
    Parameters
    ----------
    vec1: list
        first vector
    
    vec2: list
        second vector
    
    Returns
    -------
    angle: float
        angle in degrees
    
    Notes
    -----
    Angle is rounded to two decimal points
    
    """
    return np.round(np.arccos(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))*180/np.pi, decimals=2)

def write_file(outfile, data):
    """
    Write a given dict as json file
    
    Parameters
    ----------
    outfile: string
        name of output file. `.json` will be added to the given file name
    
    data: dict
        input data dict
    
    Returns
    -------
    None
    """
    with open(".".join([outfile, "json"]), "w") as fout:
        json.dump(convert_to_dict(sys), fout, cls=NumpyArrayEncoder)
    #with open(".".join([outfile, "yaml"]), "w") as fout:
    #    yaml.unsafe_dump(convert_to_dict(sys), fout)
        
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    #not available
    info["latticeparameter"] = None
    
    #needs to be defined
    info["Occupancy"] = sys.atoms.species
    info["Element"] = np.unique(sys.atoms.species)
    info["CellVolume"] = sys.volume
    info["NumberOfAtoms"] = sys.natoms

    info["X_AxisCoordinate"] = np.array(sys.atoms.positions)[:,0]
    info["Y_AxisCoordinate"] = np.array(sys.atoms.positions)[:,1]
    info["Z_AxisCoordinate"] = np.array(sys.atoms.positions)[:,2]

    #available 
    info["FirstAxisComponent"] = sys.box[0]
    info["SecondAxisComponent"] = sys.box[1]
    info["ThirdAxisComponent"] = sys.box[2]
    info["LatticeParameterLengthA"] = sys.boxdims[0]
    info["LatticeParameterLengthB"] = sys.boxdims[1]
    info["LatticeParameterLengthC"] = sys.boxdims[2]
    
    info["LatticeParameterAngleAlpha"] = get_angle(sys.box[0], sys.box[1])
    info["LatticeParameterAngleBeta"] = get_angle(sys.box[1], sys.box[2])
    info["LatticeParameterAngleGamma"] = get_angle(sys.box[2], sys.box[0])    
    
    return info

## POSCAR

In [14]:
sys = System()
sys.read_inputfile("al_data/Al.poscar", format="poscar")

In [15]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))

## LAMMPS - dump

In [16]:
sys = System()
sys.read_inputfile("al_data/Al.dump")

In [17]:
convert_to_dict(sys)
write_file("dump2", convert_to_dict(sys))

## CIF/ ASE

In [18]:
from ase.io import read

In [19]:
aseobj = read("al_data/Al.cif", format="cif")

In [20]:
sys = System()
sys.read_inputfile(aseobj, format="ase")

In [21]:
convert_to_dict(sys)
write_file("dump3", convert_to_dict(sys))

In [22]:
data = convert_to_dict(sys)

## Creating structure from pyscal

In [40]:
import pyscal.crystal_structures as pcs 

In [43]:
atoms, box = pcs.make_crystal(structure="fcc", lattice_constant=3)

In [47]:
atoms.positions

[[0.0, 0.0, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5], [0.5, 0.5, 0.0]]

In [44]:
sys = System()
sys.box = box
sys.atoms = atoms

In [45]:
convert_to_dict(sys)

{'latticeparameter': None,
 'Occupancy': [1, 1, 1, 1],
 'Element': array([1]),
 'CellVolume': 729.0,
 'NumberOfAtoms': 4,
 'X_AxisCoordinate': array([0. , 0.5, 0. , 0.5]),
 'Y_AxisCoordinate': array([0. , 0. , 0.5, 0.5]),
 'Z_AxisCoordinate': array([0. , 0.5, 0.5, 0. ]),
 'FirstAxisComponent': [3.0, 0, 0],
 'SecondAxisComponent': [0, 3.0, 0],
 'ThirdAxisComponent': [0, 0, 3.0],
 'LatticeParameterLengthA': 9.0,
 'LatticeParameterLengthB': 9.0,
 'LatticeParameterLengthC': 9.0,
 'LatticeParameterAngleAlpha': 90.0,
 'LatticeParameterAngleBeta': 90.0,
 'LatticeParameterAngleGamma': 90.0}

## TODO: pyscal side

- Take a list of elements (optionally) 
- Lattice constant is not multipled to structure creation
- Units should always be Angstroms
- Lattice constant is wrong in the above example ; should be 3 - but 9 is used
- Always use Cartesian coordinates

RDFLIB

In [23]:
from rdflib import Graph, Literal, Namespace, XSD, RDF, RDFS, URIRef, BNode, SDO
from rdflib import term

In [30]:
CSO = Namespace("https://purls.helmholtz-metadaten.de/disos/cso/")
MDO = Namespace("https://w3id.org/mdo/structure/1.1/")

In [31]:
g = Graph()
#g.bind("cso", CSO)
#g.bind("mdo", MDO)

In [32]:
Coordinate_Vector = URIRef("https://w3id.org/mdo/structure/CoordinateVector")
Atom = URIRef("https://w3id.org/mdo/structure/Atom")

In [33]:
g.add((Coordinate_Vector, MDO.X_axisCoordinate, Literal(data["X_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Y_axisCoordinate, Literal(data["Y_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Z_axisCoordinate, Literal(data["Z_AxisCoordinate"])))
g.add((Atom, MDO.))

<Graph identifier=N18dead145d324084a8c8c8cb6148e1e7 (<class 'rdflib.graph.Graph'>)>

In [36]:
with open("dump-ld.json", "w") as fout:
    fout.write(g.serialize(format="json-ld"))

In [39]:
print(g.serialize(format="json-ld"))

[
  {
    "@id": "https://w3id.org/mdo/structure/CoordinateVector",
    "https://w3id.org/mdo/structure/1.1/X_axisCoordinate": [
      {
        "@value": "[0.         0.         2.01946484 2.01946484]"
      }
    ],
    "https://w3id.org/mdo/structure/1.1/Y_axisCoordinate": [
      {
        "@value": "[0.         2.01946484 0.         2.01946484]"
      }
    ],
    "https://w3id.org/mdo/structure/1.1/Z_axisCoordinate": [
      {
        "@value": "[0.         2.01946484 2.01946484 0.        ]"
      }
    ]
  }
]
